In [1]:
import numpy
import scipy.io.wavfile
from scipy.fftpack import dct

In [8]:
sample_rate, signal = scipy.io.wavfile.read('./datasets//OSR_us_000_0010_8k.wav')  # File assumed to be in the same directory
signal = signal[0:int(3.5 * sample_rate)]  # Keep the first 3.5 seconds
signal

array([ -919, -1314, -1049, ...,   148,   136,   120], dtype=int16)

In [11]:
pre_emphasis = 0.97
emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
emphasized_signal

array([-919.  , -422.57,  225.58, ...,  -12.05,   -7.56,  -11.92])

In [13]:
frame_size = 0.025
frame_stride = 0.01

frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
signal_length = len(emphasized_signal)
frame_length = int(round(frame_length))
frame_step = int(round(frame_step))
num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

pad_signal_length = num_frames * frame_step + frame_length
z = numpy.zeros((pad_signal_length - signal_length))
pad_signal = numpy.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
frames = pad_signal[indices.astype(numpy.int32, copy=False)]

In [14]:
frames *= numpy.hamming(frame_length)

In [16]:
NFFT = 512
mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum

In [18]:
nfilt = 40
low_freq_mel = 0
high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)

fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
for m in range(1, nfilt + 1):
    f_m_minus = int(bin[m - 1])   # left
    f_m = int(bin[m])             # center
    f_m_plus = int(bin[m + 1])    # right

    for k in range(f_m_minus, f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
    for k in range(f_m, f_m_plus):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
filter_banks = numpy.dot(pow_frames, fbank.T)
filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
filter_banks = 20 * numpy.log10(filter_banks)  # dB

In [20]:
num_ceps = 12
mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13

In [22]:
cep_lifter = 22
(nframes, ncoeff) = mfcc.shape
n = numpy.arange(ncoeff)
lift = 1 + (cep_lifter / 2) * numpy.sin(numpy.pi * n / cep_lifter)
mfcc *= lift  #*

In [23]:
filter_banks -= (numpy.mean(filter_banks, axis=0) + 1e-8)


In [24]:
mfcc -= (numpy.mean(mfcc, axis=0) + 1e-8)

In [25]:
mfcc

array([[ -40.33533033,    6.27482034,   43.1938621 , ...,  -33.93320469,
         138.98679808,  158.49102169],
       [ -29.19878324,   -6.78744034,   97.2594022 , ...,  -82.60289949,
         -71.34431107,  196.73614668],
       [ -52.42728184,   11.89564837,   91.04224801, ...,   57.43195458,
         130.20282271,  270.55689551],
       ...,
       [  15.64720279,   45.08601383,   58.17261452, ...,   31.12394198,
          76.86548274, -173.84751252],
       [  23.71241102,   55.34729223,   66.45170706, ...,   34.34825596,
          84.75858549,  -50.52163398],
       [  11.67588367,   85.52352437,  100.46498648, ...,    4.00612504,
         270.3087127 ,  100.60571627]])